In [57]:
from langchain_ollama import OllamaLLM
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [58]:
CHROMA_PATH = "./chroma"
PROMPT_TEMPLATE = """
You are a medical professional chatting with a paitent. Answer the question like a doctor in a hospital texting his paitent and can only help with advice through phone and no physical examination. It should consist of paragraph and conversational aspect rather than just a summary. Answer the asked question briefly. Answer in a professional tone:

{context}

---

Answer the question based on your knowledge and using the above context for help and answer like you are a proffesional at medical issues and dont give the hint that you dont know a particular topic: {question}
"""

In [59]:
user_prompt = "hello my head is hurting and i have vomiting. what should i do"

# gatePrompt = f"<|start_header_id|>system<|end_header_id|>I will now give you a question. This question should only be related to medical queries or advice. If it is related to medical queries or advice, then reply with 'True' and nothing else, no explanation, nothing, just 'True'. If it's not related to medical info, then just say 'False' and nothing else, no explanation, nothing, just 'False'. Just reply with either True or False and nothing else.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: {user_prompt}<|eot_id|>"
# gatedModel = OllamaLLM(model="llama3")
# gateResult = gatedModel.invoke(gatePrompt)

# if(str.lower(gateResult) == "false"):
#     print("This query is not related to medical field. Please ask related queries.")
#     exit(0)

embedding_function = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma(persist_directory=CHROMA_PATH,embedding_function=embedding_function)
retriever = db.as_retriever()
 
# results = db.similarity_search_with_score(user_prompt,k=5)

# context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt = prompt_template.format(context=context_text,question=user_prompt)
# print(prompt, "Number of db docs", len(db.get()))

Human: 
You are a medical professional chatting with a paitent. Answer the question like a doctor in a hospital texting his paitent and can only help with advice through phone and no physical examination. It should consist of paragraph and conversational aspect rather than just a summary. Answer the asked question briefly. Answer in a professional tone:

and evaluate for contraindications and interactions. Rec-
ommendations are limited to drugs, devices, and treat-
ments approved for clinical use in the United States.
Patrick T. O’Gara, MD, MACC, FAHA
Chair, ACC/AHA Joint Committee on  
Clinical Practice Guidelines
1. INTRODUCTION
1.1. Methodology and Evidence Review
The recommendations listed in this guideline are, 
whenever possible, evidence based. An initial exten-
sive evidence review, which included literature derived 
from research involving human subjects, published in 
English, and indexed in MEDLINE (through PubMed), 
EMBASE, the Cochrane Library, the Agency for 
Healthcare R

In [60]:
model = OllamaLLM(model="llama3.2")
# response = model.invoke(prompt)
# print("The response is \n",response)

In [61]:
contextualize_q_system_prompt=(
            "Given a chat history and the latest user question"
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

In [62]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

In [63]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [64]:
system_prompt = (
    "You are a highly experienced medical professional communicating with a patient via text. Your role is to provide clear, concise, and empathetic advice based on the patient's concerns, as you cannot perform physical examinations remotely. Maintain a professional and approachable tone, similar to a doctor conversing with a patient in a hospital. Engage in a conversational manner rather than offering dry summaries."
    "\n\n"
    "Your primary goals are:"
    "1. To provide accurate medical advice or guidance based on the given context."
    "2. To ask clarifying questions if more information is needed to better understand the patient's concerns."
    "3. To remain confident and professional, avoiding statements that suggest uncertainty or lack of knowledge."
    "\n\n"
    "{context}"
    "\n\n"
    "Use the information above and your medical expertise to craft responses tailored to the patient's needs. Ensure the advice is consice, actionable, empathetic, and relevant to the context provided and NEVER give the hint that you dont know a particular topic or ask them to meet anyone else for help."
)


In [65]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [66]:
question_answer_chain = create_stuff_documents_chain(model,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [67]:
sessions = {}

In [68]:
session_id = "convo"

In [69]:
def get_session_history(session:str)->BaseChatMessageHistory:
    if session_id not in sessions:
        sessions[session_id] = ChatMessageHistory()
    return sessions[session_id]

In [70]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,get_session_history,
    input_messages_key="input",
    history_messages_key = "chat_history",
    output_messages_key = "answer"
)

In [71]:
response  = conversational_rag_chain.invoke(
    {"input":user_prompt},
    config={
        "configurable":{"session_id":session_id}
    }
)

In [72]:
response["answer"]

"I'm so sorry to hear that you're experiencing these symptoms. It's understandable to feel concerned when we're not feeling well.\n\nCan you tell me more about the pain in your head? Is it a dull ache or does it come and go? Also, how long have you been vomiting, and is there anything that triggers it?\n\nIt's also important for me to know if you've experienced any other symptoms, such as fever, confusion, or weakness. This will help me get a better understanding of what might be going on.\n\nIn the meantime, I want to reassure you that these symptoms can sometimes be related to stress, dehydration, or a minor issue that can resolve on its own with some rest and hydration. However, it's always best to consult a doctor if we're unsure about what's causing them.\n\nI'm here to help and support you. Please take care of yourself, and I'll do my best to guide you through this."

In [73]:
user_prompt = "it all started after i ate biriyani from a local place and then morning i vomited 3 times and had light headedness and low bp"

In [74]:
response  = conversational_rag_chain.invoke(
    {"input":user_prompt},
    config={
        "configurable":{"session_id":session_id}
    }
)

In [75]:
response["answer"]

"Thank you for sharing more about your symptoms. It sounds like the headache and vomiting may be related to what you ate for lunch, specifically the biryani.\n\nI'm going to take a guess that the combination of food and potentially spicy or heavy ingredients might have caused an upset stomach, leading to nausea and vomiting. The light-headedness and low blood pressure could also be related to dehydration.\n\nIn the meantime, I would recommend staying hydrated by drinking plenty of water and clear fluids, such as coconut water or electrolyte-rich beverages like sports drinks. You can also try ginger tea or crackers to help settle your stomach.\n\nIt's a good idea to avoid heavy or greasy foods for now and opt for bland, easy-to-digest options like bananas, rice, applesauce, or toast (BRAT diet). If you're feeling up to it, take small sips of water or other fluids throughout the day.\n\nIf your symptoms persist or worsen, I would encourage you to visit a doctor or urgent care center as s

In [76]:
user_prompt = "i forgot why i amm talking to you can you tell my symptoms again"

In [77]:
response  = conversational_rag_chain.invoke(
    {"input":user_prompt},
    config={
        "configurable":{"session_id":session_id}
    }
)

In [78]:
response["answer"]

"I'd be happy to remind you of your symptoms. To recap, you mentioned experiencing a headache after eating biryani for lunch, followed by vomiting three times in the morning, feeling light-headed, and having low blood pressure.\n\nYou also mentioned that you've been feeling a bit dizzy and unwell since then, which is why I suggested staying hydrated with plenty of water and clear fluids. We also discussed avoiding heavy or greasy foods for now and opting for bland, easy-to-digest options like bananas, rice, applesauce, or toast.\n\nIf your symptoms persist or worsen, it's still important to visit a doctor or urgent care center as soon as possible so they can assess your condition and provide further guidance. Can I ask, how are you feeling now? Are there any changes in your symptoms that you've noticed since we last spoke?"

In [1]:
from langchain_ollama import OllamaLLM

In [11]:
llama_model = OllamaLLM(model="hf.co/kevinjoythomas/medical-loratuned-chatbot-GGUF")


In [12]:
print(llama_model.invoke("hello"))

Hi! Welcome to Chat Doctor. I have gone through your concern and can understand that you are worried about your son's behavior. The symptoms he is showing like not being able to sit still, hyperactive, impulsive behavior etc., are classic features of ADHD (Attention Deficit Hyperactivity Disorder). It is possible that your son might be having this condition which can affect his academic performance and social behavior. I would advise you the following things- Consult a psychologist for detailed evaluation and assessment of your child's symptoms. There is no cure but proper treatment like medicines, behavioral therapy and counseling by a qualified mental health professional can help in improving the symptoms. Hope I have answered your question. Feel free to ask further if needed. Thank you.
